# Calculation
A lot of the design work has been done in the [Zombocalyps-project](https://github.com/LieBtrau/zombocalypse-emcomms/blob/main/amplifier/amplifier.ipynb).

## Class E amplifier filter topology
The most common way is to use a LC-series band pass filter followed by a notch filter.  The notch-filter is either a T-match or a Pi-match filter. The notch filter is used to remove the second harmonic from the output signal. 
The T-match filter requires larger inductors, which lead to more losses.

There are two ways to design the filter: either using the KD9PDP method or the NM0S method. 

Both approaches yield approximate component values.  Simulation is required to fine-tune the filter.  The [Class-E RF Power Amplifiers, Nathan O. Sokal (WA1HQC)](https://people.physics.anu.edu.au/~dxt103/160m/class_E_amplifier_design.pdf) paper explains what capacitor values to tweak based on the voltage waveforms of the drain of the MOSFET.

### KD9PDP LC-series band pass filter with Pi-match notch-filter
<img src="./images/class_e_pi_filter.png" width="500"/>

From [Search for the Minimalist QRP Class E Amplifier Filter Typology, Scott Howard (KD9PDP)](https://raw.githubusercontent.com/KD9PDP/Minimalist-Class-E/master/PDF/minimalist_class_e.pdf)


In [23]:
# Calculations for LC-series band pass filter with Pi-match notch-filter according to KD9PDP

import math

# Design parameters
VCC = 12.0  # V
Pout = 5   # W
f = 13.56e6   # Hz
R1 = 50    # Ohm

Rl = 0.58 * VCC**2 / Pout  # Ohm, load seen by the MOSFET drain
Q = 3 + math.sqrt(Rl/5 - 1)

print("Q = %.2f" % Q)
print("Rl = %.2f Ohm" % Rl)

ω = 2*math.pi*f  # rad/s
L1 = 2.84 * Rl / ω          # H, minimum value for L1
C1 = 1 / (5.4466 * ω * Rl)  # F
Cf = 1 / (Q * ω * Rl)      # F
Ltot = (Q + 1.1525) * Rl / ω  # H
C2 = (Q-3) / (ω * Rl)         # F
L2 = 15 * Q / (4 * ω)       # H
C4 = 1 / (15*Q*ω)        # F
C3 = 3 / (50*ω)          # F

print("L1 = %.2f uH" % (L1*1e6))
print("C1 = %.2f pF" % (C1*1e12))
print("Cf = %.2f pF" % (Cf*1e12))
print("Ltot = %.2f uH" % (Ltot*1e6))
print("C2 = %.2f pF" % (C2*1e12))
print("L2 = %.2f uH" % (L2*1e6))
print("C4 = %.2f pF" % (C4*1e12))
print("C3 = %.2f pF" % (C3*1e12))

Q = 4.53
Rl = 16.70 Ohm
L1 = 0.56 uH
C1 = 129.01 pF
Cf = 155.11 pF
Ltot = 1.11 uH
C2 = 1075.03 pF
L2 = 0.20 uH
C4 = 172.73 pF
C3 = 704.23 pF


After simulation, new values were found for C1 and Cf:
* C1 : 129.01pF → 146pF
* Cf : 155.11pF → 158pF

### NM0S LC-series band pass filter with Pi-match notch-filter
[Class E Power Amplifiers for QRP, David Cripe (NM0S)](http://www.amfone.net/Amforum/index.php?action=dlattach;topic=35824.0;attach=41753)

<img src="./images/NM0S_Class-E.png" width="500"/>

C1 must be mounted as close as possible to the transistor to reduce ringing due to the parasitic inductance of the traces.

C1, C2 and L2 can be 10% tolerance. L1 and L3 should be 5% tolerance. 
C3, C4, C5 and L3 should be within 5% tolerance.

In [24]:
import math
P = 5  # W, desired output power
B = 12 # V, supply voltage
F = 13.56e6 # Hz, operating frequency

Rds = 0.2 # Ohm, MOSFET on-resistance
Coss = 0 # F, output capacitance of the MOSFET

R = 0.32 * B**2 / P - 1.5 * Rds # Ohm, load resistance
C1 = 0.19 / (2*math.pi*F*R) - Coss # F, shunt capacitance
C2_min = 1 / (4 * math.pi * F * R) # series capacitance
C2_max = 1 / (2 * math.pi * F * R) # series capacitance
C2 = (C2_min + C2_max) / 2
L2 = ( 1.8 * R + 1 / ( 2 * math.pi * F * C2 )) / ( 2 * math.pi * F )
C3 = C5 = 1 / ( 2 * math.pi * F * math.sqrt(R*50))
L3 = 0.75 * math.sqrt(R*50) / ( 2 * math.pi * F)
C4 = C3 / 3
L1 = 15 * L2

print("C1 = %.2f pF" % (C1*1e12))
print("C2 = %.2f pF" % (C2*1e12))
print("L2 = %.2f uH" % (L2*1e6))
print("C3 = C5 = %.2f pF" % (C3*1e12))
print("C4 = %.2f pF" % (C4*1e12))
print("L3 = %.2f uH" % (L3*1e6))
print("L1 = %.2f uH" % (L1*1e6))

C1 = 250.12 pF
C2 = 987.31 pF
L2 = 0.33 uH
C3 = C5 = 555.89 pF
C4 = 185.30 pF
L3 = 0.19 uH
L1 = 4.92 uH


LTSpice simulation leads to the following component values changes:
* C1 : 250.12pF → 307pF
* C2 : 987.31pF → 690pF

## Comparison of the two design approaches
| Parameter | KD9PDP | NM0S |
| --- | --- | --- |
| Output power [W] | 4.57 | 7.93 |
| Input current RMS [mA]| 394 | 670 |
| Efficiency [%] | 96.6 | 98.6 |
| C1 [pF] | 146 | 307 |
| max. Vds [V] | 42 | 43 |
| Vpp(C2) | 113 | 56 |
| L2 [nH] | 1110| 330 |
| I(L2) [mA] | 509 | 905 |
| Vpp(C3) | 25 | 24.8 |
| Vpp(C4) | 62 | 63 |
| Vpp(C5) | 43 | 56 |
| L3 [nH] | 200 | 190 |
| I(L3) [mA] | 1300 | 1366 |

## Conclusion
We will use the NM0S design approach:
* ✅ The output power is higher and it's easier to reach the 5W target.
* ✅ Efficency is higher.
* ✅ C1 is higher.  The unwanted effect of Coss, which varies with Vds, is reduced.
* ✅ Peak to peak voltage over C2 is lower, which allows for a smaller capacitor.
* ✅ The inductor L2 is smaller, which reduces losses.
* ❌ The current through L2 is higher, which increases the losses.

All other parameters are similar.  Both designs have harmonics below -43dBc.

## Advice from the Infineon AN_1803 application note
<img src="./images/principle_schematic_class_E.png" width="500px"/>

1. Keep the impedance of L2 and C2 as low as possible to minimize the losses.
2. Q must be > 2.5
3. Check voltage range and current for C2 -> high voltage NP0 or C0G capacitor
4. In correctly tuned operation Vds = 3.56 * Vdd.  Incorrectly tuned operation can lead spikes up to 7 * Vdd.  Protect your transistor against that.
5. Keep Coss low.  Transistor with high Vds breakdown voltage have higher Coss.
6. Choose C1 >> Coss, because Coss is Vds dependent.
7. Coss should be as flat as possible over the Vds range.
7. Most of the loss occurs in the gate driver.  Lower Qg is more important than lower Rds_on.

In [25]:
import math
f = 13.56e6 # Switching frequency [Hz]
Pout = 5 # Output power [W]
Vdd = 12 # Supply voltage [V]
Vcc = 3.3 # Gate driver voltage [V]

# MOSFET parameters
# IRLML2060TRPBF
Qg = 0.67e-9 # Gate charge [C], from MOSFET datasheet
Rds_on = 0.64 # On-resistance of the MOSFET [Ohm], from MOSFET datasheet

# IRLML0060TRPBF
Qg = 2.5e-9 # Gate charge [C], from MOSFET datasheet
Rds_on = 0.116 # On-resistance of the MOSFET [Ohm], from MOSFET datasheet

# Input impedance of the matching network (Infineon AN-1803, eq. 13)
Z_tx_in = Vdd**2 / Pout # Input impedance of the matching network [Ohm]

# L1min (Infineon AN-1803, eq. 13)
L1_min = (math.pi**2 + 4) * Z_tx_in / f # [H]
print(f"L1min: {L1_min*1e6:.2f} uH")

# Maximum value for C1+Coss (Infineon AN-1803, eq. 20)
C1_Coss = 0.1836 / (2*math.pi*f*Z_tx_in) # [F]
print(f"Maximum value for C1+Coss: {C1_Coss*1e12:.2f} pF")

# Breakdown voltage of the MOSFET (Infineon AN-1803, eq. 19)
Vbr_dss = 3.56 * Vdd * 1.4 # [V], 40% safety margin
print(f"Breakdown voltage of the MOSFET: {Vbr_dss:.2f} V")

# Peak MOSFET current (Infineon AN-1803, eq. 22)
I_rms = math.sqrt(Pout / Z_tx_in) # [A]
I_peak = math.sqrt(2) * I_rms # [A]
print(f"Peak MOSFET current: {I_peak:.2f} A")

# MOSFET conduction losses (Infineon AN-1803, eq. 23)
P_nmos = (I_rms/2)**2 * Rds_on # [W]
print(f"MOSFET conduction losses: {P_nmos:.2f} W")

# Gate driver losses (Infineon AN-1803, eq. 24)
Pgate = Vcc * Qg * f # [W]
print(f"Gate driver losses: {Pgate:.2f} W")

L1min: 29.46 uH
Maximum value for C1+Coss: 74.82 pF
Breakdown voltage of the MOSFET: 59.81 V
Peak MOSFET current: 0.59 A
MOSFET conduction losses: 0.01 W
Gate driver losses: 0.11 W


# Transistor selection
* FET-type : N-channel
* Vdss > Vdd * 3.56 * 1.4 (safety factor) = 60V
* Id > Id_max * 1.4 (safety factor) = 1.68A -> 2A (Id_max from simulation is 1.2A.)
* Rds_on < 0.3 Ohm at Vgs = 4.5V and Id = 2A
* Vgs < 2.5V
* Qg < 4nC
* Cost < €1
* Availability : JLCPB
* Package : SOT-23 or similar
* Coss @ 10V < 160pF (Coss_max from simulation is << 160pF.)

## Candidates
| Part Number       | Coss [pF] | Vgs [V]     | Qg [nC]    | Rds_on [mOhm]  | Availability          |
|-------------------|-----------|-------------|------------|----------------|-----------------------|
| IRLML2060TRPBF    | 18        | 1.0-2.5     | 0.67       | 640            | JLCPCB : OK           |
| IRLML0060TRPBF    | 50        | 1.0-2.5     | 2.5        | 116            | JLCPCB : OK           |
| SQ2364EES-T1_GE3  | 40        | 0.46-1.0    | 2-2.5      | 245            | JLCPCB : OK           |
| SI2324A           | 25        | 1.0-2.0     | 4          | 300            | JLCPCB : OK           |
| RQ5L020SNTL       | 50        | 1.0-2.5     | 2.7        | -              | not available at JLCPCB|
| RSR020N06         | 50        | 1.0-2.5     | 2.7        | -              | not available at JLCPCB|

In [26]:
import math
Vgs = 5 # Gate-source voltage [V]

# IRLML0100
Rg = 1.3 # Internal gate resistance [Ohm], add output resistance of the driver here
Ciss = 290e-12 # Input capacitance [F], at Vds=0V (see datasheet, fig.5)
Vth = 2.5 # Threshold voltage [V]
Vgp = 2.8 # Miller plateau voltage [V] (it's the value of the horizontal line in the gate charge curves in the datasheet, fig.6)

# Turn-on delay
td_on = Rg * Ciss * math.log(Vgs/(Vgs-Vgp)) # t2, eq. 12 = t1 + t_ir = eq. 11 + eq. 17
print(f"Turn-on delay: {td_on*1e12:.2f} ps")

# Turn-off delay
td_off = Rg * Ciss * math.log(Vgs/Vgp) # t4, eq. 14
print(f"Turn-off delay: {td_off*1e12:.2f} ps")

# Turn-off delay (alternative method)
td_off = Rg * Ciss * math.log(Vgs/(Vth)) # t4 + t6, eq. 14 + eq. 16
print(f"Turn-off delay (alternative method): {td_off*1e12:.2f} ps")

Turn-on delay: 309.51 ps
Turn-off delay: 218.59 ps
Turn-off delay (alternative method): 261.32 ps


## Inductor criteria
Use smallest inductor value possible to minimize core loss.
* DC resistance
* AC resistance (skin effect, proximity effect)
* core loss (frequency dependent) : iron powder core
* RF-choke can be ferrite material

## Gate driver
* See Tru-SDX schematic for driver circuitry.
* 74ABT : 4.5V-5.5V,  IOH / IOL = -32 / +64 mA
* 74LVC : 1.65V-5.5V, IOH / IOL = -32 / +32 mA

* 74LVC1G08SE-7 : AND-gate
* 74LVC1G00SE-7 : NAND-gate

# Protection circuitry
* see Infineon AN_1803, fig. 29.
* see [Class E Amplifiers, Dan Tayloe (N7VE)](http://norcalqrp.org/files/Class_E_Amplifiers.pdf)

# References
* [FesZ Electronics : Class E RF amplifier - Basics (1/3)](https://www.youtube.com/watch?v=Tgrakttus3c)
* [Class-E power amplifier design for wireless power transfer, Infineon AN_1803_PL16_1803_210431](https://www.infineon.com/dgdl/Infineon-Wireless_power_transfer_class_E_power_amp-AN-v01_02-EN.pdf)